In [1]:
import tensorflow

In [2]:
pip install keras-tuner

  Using cached keras_tuner-1.0.3-py3-none-any.whl (96 kB)
  Using cached kt_legacy-1.0.4-py3-none-any.whl (9.6 kB)
  Using cached scipy-1.5.4-cp36-cp36m-win_amd64.whl (31.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
print(tf.__version__)

2.6.0


In [6]:
fashion_mnist=keras.datasets.fashion_mnist
print(fashion_mnist)

<module 'keras.api._v2.keras.datasets.fashion_mnist' from 'C:\\Users\\vinee\\.conda\\envs\\SANJU_ME\\lib\\site-packages\\keras\\api\\_v2\\keras\\datasets\\fashion_mnist\\__init__.py'>


In [7]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 2s 0us/step


In [8]:
train_images=train_images/255.0
test_images=test_images/255.0

In [10]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [17]:
train_images[0].shape

(28, 28, 1)

In [18]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [19]:
train_images[0].shape

(28, 28, 1)

In [20]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [22]:
import keras_tuner
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [23]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [24]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 08m 20s]
val_accuracy: 0.9096666574478149

Best val_accuracy So Far: 0.9096666574478149
Total elapsed time: 00h 55m 40s
INFO:tensorflow:Oracle triggered exit


In [25]:
model=tuner_search.get_best_models(num_models=1)[0]

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                884768    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 913,434
Trainable params: 913,434
Non-trainable params: 0
_________________________________________________________________


In [27]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 162s 95ms/step - loss: 0.1659 - accuracy: 0.9385 - val_loss: 0.2547 - val_accuracy: 0.9133
Epoch 5/10
1688/1688 [==============================] - 156s 92ms/step - loss: 0.1284 - accuracy: 0.9520 - val_loss: 0.2573 - val_accuracy: 0.9162
Epoch 6/10
1688/1688 [==============================] - 184s 109ms/step - loss: 0.0975 - accuracy: 0.9633 - val_loss: 0.2728 - val_accuracy: 0.9183
Epoch 7/10
1688/1688 [==============================] - 166s 98ms/step - loss: 0.0725 - accuracy: 0.9726 - val_loss: 0.3180 - val_accuracy: 0.9123
Epoch 8/10
1688/1688 [==============================] - 165s 98ms/step - loss: 0.0564 - accuracy: 0.9789 - val_loss: 0.3547 - val_accuracy: 0.9177
Epoch 9/10
1688/1688 [==============================] - 162s 96ms/step - loss: 0.0441 - accuracy: 0.9843 - val_loss: 0.4001 - val_accuracy: 0.9160
Epoch 10/10
1688/1688 [==============================] - 160s 95ms/step - loss: 0.0360 - accuracy: 0.9866 - val_loss: